# Exploring Car Data with Pandas

In [1]:
# now that we have scraped data we want to clean and process it
# afterwards we will want to generate some statistics and visualizations

# first let's load libraries 

# first standard libraries
# python version
import sys
print(f"Python version: {sys.version}")
from pathlib import Path
from datetime import datetime
import re

# data manipulation
import pandas as pd
print(f"Pandas version: {pd.__version__}")

# visualization with plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
# version
print(f"Plotly version: {plotly.__version__}")

Python version: 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]
Pandas version: 2.2.2
Plotly version: 5.22.0


In [2]:
parquet_folder = Path("../data/parquet")
assert parquet_folder.exists(), "Parquet folder does not exist"

In [5]:
parquet_files = list(parquet_folder.glob("*.parquet"))
print(f"Found {len(parquet_files)} parquet files")
# if we have at least one get the latest
# sort by modification time and get the last one
parquet_files = sorted(parquet_files, key=lambda x: x.stat().st_mtime)

if len(parquet_files) > 0:
    latest_parquet = parquet_files[-1]
    print(f"Latest parquet file: {latest_parquet}")
    # read the parquet file
    df = pd.read_parquet(latest_parquet)
    print(f"Dataframe shape: {df.shape}")
    print(f"Dataframe columns: {df.columns}")
    display(df.head())
else:
    print("No parquet files found, please run the scraping notebook first")

Found 1 parquet files
Latest parquet file: ..\data\parquet\all_cars.parquet
Dataframe shape: (24611, 8)
Dataframe columns: Index(['description', 'url', 'Modelis', 'Gads', 'Tilp.', 'Nobrauk.', 'Cena',
       'Marka'],
      dtype='object')


,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena,Marka
0,1.9 88kw. Labā tehniskā un vizuāla stāvoklī. Ļ...,https://ss.com/msg/lv/transport/cars/alfa-rome...,159,2008,1.9D,-,"1,550 €",None
1,Pārdodu vāciešu alfa romeo 1.9 Jtd tehniski la...,https://ss.com/msg/lv/transport/cars/alfa-rome...,159,2009,1.9D,177 tūkst.,"2,900 €",None
2,Alfa Romeo Stelvio\r\n2.2 dīzeļdzinējs\r\nAuto...,https://ss.com/msg/lv/transport/cars/alfa-rome...,Stelvio,2017,2.2D,158 tūkst.,"16,990 €",None
3,"Alfa Romeo Mito 1.3Jtdm 95Hp, ""Distinctive"" ap...",https://ss.com/msg/lv/transport/cars/alfa-rome...,Citi,2011,1.3D,175 tūkst.,"3,900 €",None
4,Alfa romeo 147. Automāts 2.0benzīns ekonomisks...,https://ss.com/msg/lv/transport/cars/alfa-rome...,147,2004,2.0,-,"1,499 €",None


In [6]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

df.head().style.format({'url': make_clickable})

,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena,Marka
0,1.9 88kw. Labā tehniskā un vizuāla stāvoklī. Ļoti labas vissezonas riepas uz,https://ss.com/msg/lv/transport/cars/alfa-romeo/159/gnhhf.html,159,2008,1.9D,-,"1,550 €",None
1,Pārdodu vāciešu alfa romeo 1.9 Jtd tehniski labā stāvoklī ar dažiem vizuālie,https://ss.com/msg/lv/transport/cars/alfa-romeo/159/cdocni.html,159,2009,1.9D,177 tūkst.,"2,900 €",None
2,Alfa Romeo Stelvio 2.2 dīzeļdzinējs Automātiskā 8 pakāpju ātrumkārba Park,https://ss.com/msg/lv/transport/cars/alfa-romeo/stelvio/cdxnhe.html,Stelvio,2017,2.2D,158 tūkst.,"16,990 €",None
3,"Alfa Romeo Mito 1.3Jtdm 95Hp, ""Distinctive"" aprīkojuma līmenis, ""Blu Tornado",https://ss.com/msg/lv/transport/cars/alfa-romeo/another/nbcxd.html,Citi,2011,1.3D,175 tūkst.,"3,900 €",None
4,"Alfa romeo 147. Automāts 2.0benzīns ekonomisks 5-6L 100/km, laba komplektāci",https://ss.com/msg/lv/transport/cars/alfa-romeo/147/lpoec.html,147,2004,2.0,-,"1,499 €",None


In [8]:
# unique Models
models = df['Modelis'].unique()
print(f"Unique models: {len(models)}")

Unique models: 865


In [14]:
# unique Marka
brands = df['Marka'].unique()
print(f"Unique brands: {len(brands)}")
# print all brands sorted
# convert NoneType to empty string
brands = ["" if pd.isna(b) else b for b in brands]
brands = sorted(brands)
print(brands)

Unique brands: 62
['', '-', 'Austin', 'CadillacCiti', 'CadillacDE Ville', 'CadillacEldorado', 'CadillacEscalade', 'CadillacSRX', 'CadillacSeville', 'Chevrolet Chev', 'DaewooKalos', 'DaewooLanos', 'Daihatsu', 'Geely ck', 'Gmc', 'HummerH2', 'HummerH3', 'Hyundai', 'IsuzuCiti', 'IsuzuPick Up', 'IsuzuTrooper', 'Iž kombi', 'IžKombi', 'Karma revero', 'Levc tx', 'Levc vn5', 'Lincoln', 'LincolnMark', 'LincolnNavigator', 'LincolnTown Car', 'Linkoln', 'Malagutti', 'Man', 'Maserati', 'Mazda B2500', 'Metallolom', 'Mg zt-t', 'Ora', 'Rl1100', 'Rover', 'Rover416', 'Rover75', 'Smz', 'Ssang yong', 'SsangYongActyon', 'SsangYongCiti', 'SsangYongKorando', 'SsangYongRexton', 'Styer', 'Tesla', 'Uaz hunter', 'Uaz3151', 'Uaz3303', 'Uaz452', 'Uaz469', 'Vissi', 'Yamaha', 'Zaz', 'Zaz1102', 'Zaz965', 'Zaz966', 'Zaz968']


In [16]:
# value counts of brands
brand_counts = df['Marka'].value_counts()
brand_counts

Marka
SsangYongRexton     7
CadillacEscalade    7
Uaz469              6
LincolnTown Car     5
Uaz452              4
                   ..
DaewooKalos         1
DaewooLanos         1
Zaz965              1
Ora                 1
Zaz966              1
Name: count, Length: 61, dtype: int64

In [18]:
# How many are none?
none_brands = df['Marka'].isnull().sum()
print(f"None brands: {none_brands}")

None brands: 24507


In [22]:
# let's create a function that extracts brand from url
# brand comes after cars/ and before next /
def extract_brand(url):
    if pd.isna(url):
        return None
    match = re.search(r"cars/(.*?)/", url)
    if match:
        return match.group(1)
    else:
        return None

In [23]:
# let's create a brand column
df['Brand'] = df['url'].apply(extract_brand)
# value counts
brand_counts = df['Brand'].value_counts()
brand_counts

Brand
bmw           3748
volkswagen    3186
audi          2538
volvo         2310
mercedes      1594
toyota        1421
opel          1396
ford          1000
renault        739
skoda          654
nissan         632
peugeot        600
citroen        447
hyundai        440
honda          358
kia            313
land-rover     304
lexus          275
mitsubishi     258
mazda          249
seat           214
porsche        194
subaru         175
jeep           172
mini           158
chrysler       138
fiat           138
chevrolet      129
jaguar         105
vaz             95
dacia           91
suzuki          83
saab            65
alfa-romeo      60
dodge           53
lancia          40
others          36
smart           33
gaz             28
infiniti        26
cupra           25
moskvich        17
ssangyong       15
cadillac        15
uaz             12
lincoln          9
zaz              6
hummer           5
isuzu            5
rover            4
daewoo           2
iz               1
Name: 

In [26]:
# now let's extract price from Cena
# we want to get rid of whitespace then comman and then Euro sign
def extract_price(price):
    if pd.isna(price):
        return None
    price = price.strip()
    price = price.replace(" ", "")
    price = price.replace(",", "")
    price = price.replace("€", "")
    # now try converting to float
    try:
        return float(price)
    except ValueError:
        return 0


# let's create a price column
df['Price'] = df['Cena'].apply(extract_price)
# value counts
price_counts = df['Price'].value_counts()
price_counts

Price
0.0        1473
2500.0      228
3500.0      226
4500.0      203
1500.0      201
           ... 
1455.0        1
34100.0       1
8288.0        1
22385.0       1
830.0         1
Name: count, Length: 2089, dtype: int64

In [27]:
# now let's create km column 
# we will use Nobrauk. column
# we will replace km with empty string
# we will replace tukst. with 000
# we will replace whitespace with empty string
def extract_km(km):
    if pd.isna(km):
        return None
    km = km.replace("km", "")
    km = km.replace("tūkst.", "000")
    km = km.replace(" ", "")
    # now try converting to float
    try:
        return float(km)
    except ValueError:
        return 0
    
# let's create a km column
df['Km'] = df['Nobrauk.'].apply(extract_km)
# value counts
km_counts = df['Km'].value_counts()
km_counts

Km
0.0         5176
300000.0     194
250000.0     165
280000.0     148
260000.0     144
            ... 
2.8            1
162.0          1
2.4            1
969000.0       1
5.7            1
Name: count, Length: 694, dtype: int64

In [28]:
# let's create Year column from Gads column by converting to int
def extract_year(year):
    if pd.isna(year):
        return None
    # now try converting to int
    try:
        return int(year)
    except ValueError:
        return 0

# let's create a year column
df['Year'] = df['Gads'].apply(extract_year)
# value counts
year_counts = df['Year'].value_counts()
year_counts

Year
2007    1730
2008    1626
2011    1434
2012    1368
2006    1298
        ... 
1956       1
1965       1
1968       1
1967       1
1936       1
Name: count, Length: 70, dtype: int64

In [29]:
# df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24611 entries, 0 to 24610
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   description  24611 non-null  object 
 1   url          24611 non-null  object 
 2   Modelis      24507 non-null  object 
 3   Gads         24611 non-null  object 
 4   Tilp.        24611 non-null  object 
 5   Nobrauk.     24611 non-null  object 
 6   Cena         24611 non-null  object 
 7   Marka        104 non-null    object 
 8   Brand        24611 non-null  object 
 9   Price        24611 non-null  float64
 10  Km           24611 non-null  float64
 11  Year         24611 non-null  int64  
dtypes: float64(2), int64(1), object(9)
memory usage: 2.3+ MB


In [31]:
# describe years column
df['Year'].describe()

count    24611.000000
mean      1948.640242
std        346.974225
min          0.000000
25%       2006.000000
50%       2010.000000
75%       2015.000000
max       2024.000000
Name: Year, dtype: float64

In [32]:
# how many 0 years?
zero_years = df[df['Year'] == 0].shape[0]
print(f"Zero years: {zero_years}")

Zero years: 756


In [33]:
# let's plot value counts of years
# ignore zero years
year_counts = year_counts[year_counts.index != 0]
fig = px.bar(year_counts, x=year_counts.index, y=year_counts.values, labels={'x': 'Year', 'y': 'Count'})
fig.update_layout(title_text='Value counts of years')
fig.show()

In [34]:
# let us save the cleaned dataframe
cleaned_parquet = parquet_folder / "cleaned_cars.parquet"
df.to_parquet(cleaned_parquet)


In [38]:
# df shape
print(f"Dataframe shape: {df.shape}")

Dataframe shape: (24611, 12)


In [41]:
# let us get all RAV4 cars
# this means we will filter by Modelis column or Description column
rav4_cars = df[(df['Modelis'].str.contains("RAV") )| (df['description'].str.contains("RAV", na=False))]
print(f"RAV4 cars shape: {rav4_cars.shape}")

RAV4 cars shape: (252, 12)


In [42]:
# print value counts for rav4 cars by years
rav4_year_counts = rav4_cars['Year'].value_counts()
print(rav4_year_counts)

Year
2019    29
0       22
2007    17
2006    15
2014    13
2012    12
2008    12
2011    11
2022    11
2003    11
2021    10
2013    10
2020    10
2010    10
2023     7
2015     7
2016     7
2005     6
2002     6
2017     6
2004     5
2001     4
2018     3
2009     3
2024     3
2000     2
Name: count, dtype: int64


In [44]:
# print candlestick plot for rav4 cars prices by year
rav4_cars = rav4_cars[rav4_cars['Price'] > 0]
fig = px.box(rav4_cars, x='Year', y='Price', labels={'x': 'Year', 'y': 'Price'})
fig.update_layout(title_text='RAV4 cars prices by year')
fig.show()

In [78]:
# let's creat a function that will plot a box plot for a given model given a full df
# we want color to indicate the number of cars in a given year
# title should show total number of cars
def plot_model_prices(df, model):
    model_cars = df[(df['Modelis'].str.contains(model) )| (df['description'].str.contains(model, na=False))]
    model_cars = model_cars[model_cars['Price'] > 0]
    print(f"{model} cars shape: {model_cars.shape}")


    fig = px.box(model_cars,
                  x='Year',
                y='Price',
                  labels={'x': 'Year', 'y': 'Price'}, 
                  points='all', 
                  color='Year',
                  notched=True,
                #   variable_width=True,
                  )

    fig.update_layout(title_text=f'{model} cars prices by year - Total cars: {model_cars.shape[0]}')
    fig.show()
    return model_cars

plot_model_prices(df, "RAV")
;

RAV cars shape: (220, 12)


''

In [79]:
# test with xc40
xc40_cars = plot_model_prices(df, "XC 40")

XC 40 cars shape: (29, 12)


In [80]:
xc60 = plot_model_prices(df, "XC 60")

XC 60 cars shape: (319, 12)


In [81]:
# now xc90 cars
xc90 = plot_model_prices(df, "XC 90")

XC 90 cars shape: (252, 12)


In [82]:
# now x-trail cars
xtrail = plot_model_prices(df, "X-Trail")

X-Trail cars shape: (113, 12)


In [83]:
# now qashqai cars
qashqai = plot_model_prices(df, "Qashqai")


Qashqai cars shape: (206, 12)


In [84]:
# now sorrento
sorrento = plot_model_prices(df, "Sorento")

Sorento cars shape: (41, 12)


In [85]:
# Now Forrester
forrester = plot_model_prices(df, "Forester")

Forester cars shape: (48, 12)


In [86]:
# now outback
outback = plot_model_prices(df, "Outback")

Outback cars shape: (35, 12)


In [87]:
# now CRV
crv = plot_model_prices(df, "Cr-v")

Cr-v cars shape: (115, 12)


In [88]:
# now Q3
q3 = plot_model_prices(df, "Q3")

Q3 cars shape: (24, 12)


In [100]:
# let's explore RAV4 cars - we want cars after 2012
rav4_2012 = rav4_cars[rav4_cars['Year'] >= 2012]
# sort by price
rav4_2012 = rav4_2012.sort_values('Price', ascending=False)
print(f"RAV4 cars after 2012 shape: {rav4_2012.shape}")
# make links clickable

rav4_2012.head()
rav4_2012.style.format({'url': make_clickable})

RAV4 cars after 2012 shape: (128, 12)


,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena,Marka,Brand,Price,Km,Year
18301,Toyota Rav4 Plug-in Hybrid Suv 2.5 Hybrid Dynamic Force (306 hp) Awd E-Cvt P,https://ss.com/msg/lv/transport/cars/toyota/rav-4/cbongn.html,RAV 4,2023,2.5H,2.5 tūkst.,"57,990 €",None,toyota,57990.000000,2.500000,2023
18630,11000Km. Uzņēmums Pārdod Toyota Rav4 Plug-in Hybrid Suv 2.5 Hybrid Dynamic F,https://ss.com/msg/lv/transport/cars/toyota/rav-4/cdhbpj.html,RAV 4,2022,2.5H,-,"54,700 €",None,toyota,54700.000000,0.000000,2022
17546,"Toyota Bz4X Elektro, 150.kw, 204.zs, Automāts. Cena ar Pvn. 11000.km",https://ss.com/msg/lv/transport/cars/toyota/rav-4/bjmpmc.html,RAV 4,2023,E,-,"49,500 €",None,toyota,49500.000000,0.000000,2023
18546,"Jauns auto, ātra piegāde. Rav4 Hybrid Suv 2.5 Hybrid Dynamic Force (222 hp)",https://ss.com/msg/lv/transport/cars/toyota/rav-4/cddxil.html,RAV 4,2024,2.5H,10,"46,600 €",None,toyota,46600.000000,10.000000,2024
18660,"Pārdod showroom auto, pieejams uzreiz. Rav4 2.5 Hybrid Dynamic Force (218 hp",https://ss.com/msg/lv/transport/cars/toyota/rav-4/bbnnjl.html,RAV 4,2024,2.5H,85,"45,150 €",None,toyota,45150.000000,85.000000,2024
18226,"Toyota Rav4 Suv 2.5 Hybrid Dynamic Force (222 hp) 4Wd E-Cvt Luxury, automāts",https://ss.com/msg/lv/transport/cars/toyota/rav-4/ccpjjg.html,RAV 4,2023,2.5H,8.5 tūkst.,"39,990 €",None,toyota,39990.000000,8.500000,2023
17577,"Toyota Rav4 Hybrid Suv 2.5 Hybrid Dynamic Force (218 hp) E-Cvt Luxury Plus,",https://ss.com/msg/lv/transport/cars/toyota/rav-4/cdbmxf.html,RAV 4,2023,2.5H,7.5 tūkst.,"38,990 €",None,toyota,38990.000000,7.500000,2023
18337,Rav4 2.5 Hybrid Dynamic Force (218 hp) E-Cvt Luxury Limited Cenā iekļauti,https://ss.com/msg/lv/transport/cars/toyota/rav-4/cbpkog.html,RAV 4,2024,2.5H,10,"38,420 €",None,toyota,38420.000000,10.000000,2024
17637,"Sakarā ar darba auto iegūšanu tiek pārdots principā jauns ļoti ekonomisks, i",https://ss.com/msg/lv/transport/cars/toyota/rav-4/cdcdmf.html,RAV 4,2023,2.5H,12 tūkst.,"38,200 €",None,toyota,38200.000000,12000.000000,2023
18228,"Toyota Rav4 Hybrid Suv 2.5 Hybrid Dynamic Force (218 hp) E-Cvt Luxury Plus,",https://ss.com/msg/lv/transport/cars/toyota/rav-4/ccpjld.html,RAV 4,2023,2.5H,24 tūkst.,"37,990 €",None,toyota,37990.000000,24000.000000,2023


In [94]:

print(f"RAV4 cars after 2012 shape: {rav4_2012.shape}")
# let's plot prices
fig = px.box(rav4_2012, x='Year', y='Price', 
             labels={'x': 'Year', 'y': 'Price'},
             # custom hover data
            hover_data=['description', 'Price', 'Km', 'url'],
             points='all', 
             color='Year')
fig.update_layout(title_text='RAV4 cars prices after 2012 by year')
# show each year in x ticks
fig.update_xaxes(tickmode='array', tickvals=rav4_2012['Year'].unique())
fig.show()

RAV4 cars after 2012 shape: (128, 12)


In [101]:
# now let's explore xc60
xc60_2012 = xc60[xc60['Year'] >= 2012]
# sort by price
xc60_2012 = xc60_2012.sort_values('Price', ascending=False)
print(f"XC60 cars after 2012 shape: {xc60_2012.shape}")
# make links clickable
xc60_2012.style.format({'url': make_clickable})


XC60 cars after 2012 shape: (202, 12)


,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena,Marka,Brand,Price,Km,Year
22656,Auto jauns pirkts un apkopi gājis te pat pie Mūsa Motors. 23 modeļa gads (,https://ss.com/msg/lv/transport/cars/volvo/xc60/cdmomm.html,XC 60,2022,2.0D,26 tūkst.,"45,000 €",None,volvo,45000.000000,26000.000000,2022
24202,T8 Plugin Hybrid. Awd. 2L/100km 403zs Bowers&wilkins. Kamera. Webasto. Head,https://ss.com/msg/lv/transport/cars/volvo/xc60/cxgpim.html,XC 60,2019,2.0H,94 tūkst.,"43,900 €",None,volvo,43900.000000,94000.000000,2019
23699,"2021, Elekto/benzīns, Plug in -Recharge T6, Awd 2.0l, atpakaļskatu Kamera, A",https://ss.com/msg/lv/transport/cars/volvo/xc60/cdijjh.html,XC 60,2021,2.0,-,"42,200 €",None,volvo,42200.000000,0.000000,2021
23583,"Volvo Xc60 Core, Fwd B5 2.0L Benzīns, 145Kw/197Zs Mild Hybrid, 8-pak. automā",https://ss.com/msg/lv/transport/cars/volvo/xc60/cbkdhx.html,XC 60,2022,2.0,4.5 tūkst.,"41,990 €",None,volvo,41990.000000,4.500000,2022
23260,Volvo Xc-60 Facelift Plus Dark R-Design 2.0 B4 Mild Hybrid-197 z/s. Automāts,https://ss.com/msg/lv/transport/cars/volvo/xc60/bdidcb.html,XC 60,2021,2.0D,67 tūkst.,"40,990 €",None,volvo,40990.000000,67000.000000,2021
23577,"Volvo Xc60 T8, Awd, 303Kw/412Zs, 2.0 elektrība un benzīns, 8-pak. automāti",https://ss.com/msg/lv/transport/cars/volvo/xc60/ccmhgx.html,XC 60,2019,2.0H,67 tūkst.,"39,990 €",None,volvo,39990.000000,67000.000000,2019
23190,"Volvo Xc60 B4 Awd R-design Facelift 197 z/s Auto tikko ievests, pilna servi",https://ss.com/msg/lv/transport/cars/volvo/xc60/cdkpbl.html,XC 60,2019,2.0D,91 tūkst.,"37,990 €",None,volvo,37990.000000,91000.000000,2019
22869,"Volvo Xc60 B5 Mild Hybrid, Awd, 235Hp, Latvijā nav ekspluatēta. Pirmā reģi",https://ss.com/msg/lv/transport/cars/volvo/xc60/cddnxx.html,XC 60,2020,2.0D,119 tūkst.,"36,950 €",None,volvo,36950.000000,119000.000000,2020
23927,"Hybrid/dīzelis, 143kW, atpakaļskatu Kamera, Automātiskā parkošanās sistēma,",https://ss.com/msg/lv/transport/cars/volvo/xc60/bpcggn.html,XC 60,2021,2.0D,-,"36,500 €",None,volvo,36500.000000,0.000000,2021
24041,Full option Inscription D4 Balta Pērle kr. Masāža. Bowers&wilkins. 360kamera,https://ss.com/msg/lv/transport/cars/volvo/xc60/cbenfo.html,XC 60,2018,2.0D,87 tūkst.,"35,900 €",None,volvo,35900.000000,87000.000000,2018


In [102]:
# now forester
forrester_2012 = forrester[forrester['Year'] >= 2012]
# sort by price
forrester_2012 = forrester_2012.sort_values('Price', ascending=False)
print(f"Forrester cars after 2012 shape: {forrester_2012.shape}")
# make links clickable
forrester_2012.style.format({'url': make_clickable})

Forrester cars after 2012 shape: (29, 12)


,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena,Marka,Brand,Price,Km,Year
17287,"Jaunais Subaru Forester Limited 2.5 i 134 kw 183 hp rindas Lcd monitors,",https://ss.com/msg/lv/transport/cars/subaru/forester/bincib.html,Forester,2022,2.5,-,"33,900 €",None,subaru,33900.000000,0.000000,2022
17300,Subaru Forester 2020 Ietilpīgs un Gaišs: Subaru Forester piedāvā plašu s,https://ss.com/msg/lv/transport/cars/subaru/forester/bkjddg.html,Forester,2020,2.0,42 tūkst.,"27,000 €",None,subaru,27000.000000,42000.000000,2020
17247,Subaru Forester 2.0i Hybrid (150 zs) ar automātisko pārnesumkārbu. Cena ar P,https://ss.com/msg/lv/transport/cars/subaru/forester/cckmnn.html,Forester,2020,2.0,138 tūkst.,"24,450 €",None,subaru,24450.000000,138000.000000,2020
17296,"Pārdodu mašīnu, pilna komplektācija, ir ziemas riepas, visas apkopes veiktas",https://ss.com/msg/lv/transport/cars/subaru/forester/cdhcod.html,Forester,2020,2.0H,38 tūkst.,"24,300 €",None,subaru,24300.000000,38000.000000,2020
17210,"5. paaudzes krosovers, visu riteņu piedziņa - Summetrical All Wheel Drive, k",https://ss.com/msg/lv/transport/cars/subaru/forester/fkcmo.html,Forester,2019,2.5,48 tūkst.,"22,500 €",None,subaru,22500.000000,48000.000000,2019
17258,"Pārdodu Subaru Forester E-Boxer 2.0l benzīns, 110kw/150hp Auto var apskatīt",https://ss.com/msg/lv/transport/cars/subaru/forester/cdxecx.html,Forester,2021,2.0,152 tūkst.,"21,200 €",None,subaru,21200.000000,152000.000000,2021
17271,"4x4 pilnpiedziņa. Pirkts Latvijā, no AD Rem auto. Īsts nobraukums. Patēr",https://ss.com/msg/lv/transport/cars/subaru/forester/cdjxlg.html,Forester,2018,2.0,82 tūkst.,"17,400 €",None,subaru,17400.000000,82000.000000,2018
17285,"Subaru Forester XS 2.0i/ 150 Zs jauda, Led gaismas ar līkumgaismām, 6-pakā",https://ss.com/msg/lv/transport/cars/subaru/forester/cdhnig.html,Forester,2017,2.0,120 tūkst.,"17,300 €",None,subaru,17300.000000,120000.000000,2017
17178,Pārdodu Subaru Forester 110kW 2016. Carprof garantija 8 mēnešu vai 5000 km.,https://ss.com/msg/lv/transport/cars/subaru/forester/cbmdcg.html,Forester,2016,2.0,86 tūkst.,"16,499 €",None,subaru,16499.000000,86000.000000,2016
17149,Ievests no Vācijas. 1 Saimnieks. Rūpnīcas krāsojums. Visas apkopes pie dīler,https://ss.com/msg/lv/transport/cars/subaru/forester/cdoceh.html,Forester,2018,2.0D,-,"15,950 €",None,subaru,15950.000000,0.000000,2018


In [103]:
# finally xtrail
xtrail_2012 = xtrail[xtrail['Year'] >= 2012]
# sort by price
xtrail_2012 = xtrail_2012.sort_values('Price', ascending=False)
print(f"X-Trail cars after 2012 shape: {xtrail_2012.shape}")
# make links clickable
xtrail_2012.style.format({'url': make_clickable})


X-Trail cars after 2012 shape: (49, 12)


,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena,Marka,Brand,Price,Km,Year
12958,"Nissan X-Trail e-4orce, hibrīds, 4Wd, komplektācija Tekna. Papildus opci",https://ss.com/msg/lv/transport/cars/nissan/x-trail/mkgmh.html,X-Trail,2024,1.5H,10,"45,880 €",None,nissan,45880.000000,10.000000,2024
13167,Nissan X-Trail Tekna 1.3Dig-T (160 zs) ar automātisko pārnesumkārbu. Cena ar,https://ss.com/msg/lv/transport/cars/nissan/x-trail/cdhhdg.html,X-Trail,2020,1.3,90 tūkst.,"21,200 €",None,nissan,21200.000000,90000.000000,2020
12918,"Pārdodu uzticamu auto, tehnisko apskati izgāju vienmēr pa 0, 0; ieguldījumus",https://ss.com/msg/lv/transport/cars/nissan/x-trail/cdlbdd.html,X-Trail,2019,2.0D,110 tūkst.,"21,000 €",None,nissan,21000.000000,110000.000000,2019
13183,"Nissan X-Trail. 7vietas. Automāts. Labs aprīkojums. Servisa vēsture Skandi,",https://ss.com/msg/lv/transport/cars/nissan/x-trail/bgdcfp.html,X-Trail,2020,1.3,58 tūkst.,"20,500 €",None,nissan,20500.000000,58000.000000,2020
13086,Nissan X-Trail 1.7dCi 150zs. Dīzelis. Pilna komplektācija. Nobraukums 54k. M,https://ss.com/msg/lv/transport/cars/nissan/x-trail/bnhxbx.html,X-Trail,2020,1.7D,54 tūkst.,"20,000 €",None,nissan,20000.000000,54000.000000,2020
12660,"Nissan X-Trail, Tekna komplektācija ar 1.6 Dīzeļa dzinēju un Automātisko ātr",https://ss.com/msg/lv/transport/cars/nissan/x-trail/bxmfep.html,X-Trail,2018,1.6D,57 tūkst.,"19,990 €",None,nissan,19990.000000,57000.000000,2018
12807,Nissan X-Trail 1.6D. Pvn. Pilna servisa Vēsture. Iegādājies savu auto pi,https://ss.com/msg/lv/transport/cars/nissan/x-trail/cdmkfc.html,X-Trail,2018,1.6D,119 tūkst.,"19,990 €",None,nissan,19990.000000,119000.000000,2018
12854,"Pārdodu mīlētu, labi koptu un ietilpīgu Nissan X-Trail 150zs Tekna Moonroof",https://ss.com/msg/lv/transport/cars/nissan/x-trail/bmpbil.html,X-Trail,2020,1.7D,177 tūkst.,"19,800 €",None,nissan,19800.000000,177000.000000,2020
13006,Pārdod Nissan X-Trail labā tehniskā un vizuālā stāvoklī.,https://ss.com/msg/lv/transport/cars/nissan/x-trail/bmoxbc.html,X-Trail,2020,1.3,77 tūkst.,"19,500 €",None,nissan,19500.000000,77000.000000,2020
13037,Sveiki. Pārdodas Nissan X-Trail 2020. gada modeļa automašīna. Ekonomisks tra,https://ss.com/msg/lv/transport/cars/nissan/x-trail/bcjoim.html,X-Trail,2020,1.7D,81 tūkst.,"19,350 €",None,nissan,19350.000000,81000.000000,2020
